Установка зависимостей

In [270]:
!pip install requests scikit-learn nerus numpy pandas


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Основные константы

In [271]:
import os

PATH = '.'
PREDICT_URL = 'https://caila.io/api/mlpgate/account/1000167571/model/90066/predict'
REQUEST_PATTERN = {
  "service": "",
  "model": "",
  "text": ""
}
HEADER = {
            "MLP-API-KEY": os.getenv('MLP_API_KEY'),
            "Content-Type": "application/json"
}

Оценки f1, recall, precision

In [272]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(predictions, ground_truths):
    y_true = [1 if entity in gt else 0 for gt in ground_truths for entity in predictions]
    y_pred = [1 if entity in predictions else 0 for gt in ground_truths for entity in predictions]

    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    return precision, recall, f1



Оценка скорости работы

In [273]:
import time

def measure_speed(predict_function, data):
    start_time = time.time()
    result = predict_function(data)
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time, result


Создание реквеста

In [274]:
def create_request_payload(service: str, model: str, text: str) -> dict:
    request = REQUEST_PATTERN.copy()
    request['service'] = service
    request['model'] = model
    request['text'] = text
    return request

Запрос на сервис

In [275]:
import requests

import json

def predict_function(service: str, model: str, text: str):
    print(service, model, text)
    payload = create_request_payload(service, model, text)
    response = requests.post(url=PREDICT_URL, json=payload, headers=HEADER)
    print(response)
    response_data = response.json()
    entities = json.loads(response_data.get('response', '[]')) 
    return entities

Работа с csv

In [276]:
import csv
from typing import List, Dict, Tuple

def extract_per_entities(doc):
    text, spans = doc.ner
    return [text[span.start:span.stop] for span in spans if span.type == 'PER']

def count_tokens(text: str) -> int:
    # Простейшая оценка: количество слов
    return len(text.split())

def save_to_csv(text: str, ground_truth: List[str], predictions_per_model: Dict[str, List[str]]):
    rows = []
    # Собираем данные для каждой модели
    for model, predictions in predictions_per_model.items():
        row = [text, ', '.join(ground_truth), model, ', '.join(predictions)]
        rows.append(row)

    # Записываем в CSV файл
    with open('results.csv', 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='|')
        writer.writerows(rows)



Основной скрипт теста

In [282]:
def run_evaluation(docs: List, services_and_models: List[Dict], num_docs=10) -> Tuple[List[Dict], List[Dict]]:
    metrics = []
    speed_data = []
    
    processed_count = 0  # Счетчик обработанных документов
    
    for doc in docs:
        if processed_count >= num_docs:
            break
        
        ground_truth = extract_per_entities(doc)
        
        if not ground_truth or len(doc.ner.text) > 1000:
            continue
        
        # Подсчёт токенов
        token_count = count_tokens(doc.ner.text)
        
        predictions_per_model = {}
        
        for service_model in services_and_models:
            service = service_model['service']
            model = service_model['model']
            text = doc.ner.text
            
            # Замеряем время обработки и получаем предсказания
            try:
                processing_time, predictions = measure_speed(lambda x: predict_function(service, model, x), text)
                predictions_per_model[model] = predictions

                # Оцениваем модель
                precision, recall, f1 = evaluate_model(predictions, ground_truth)
                metrics.append({
                    'service': service, 
                    'model': model, 
                    'precision': precision, 
                    'recall': recall, 
                    'f1': f1
                })
                
                speed_data.append({
                    'service': service, 
                    'model': model, 
                    'processing_time': processing_time,
                    'input_tokens': token_count,  # Добавляем количество токенов для входных данных
                    'output_tokens': token_count  # Предположим, что выходные токены равны входным (если это не так, исправьте соответствующим образом)
                })

            except Exception as e:
                print(f"Error with service {service} and model {model}: {e}")

        # Сохраняем результаты в CSV
        save_to_csv(doc.ner.text, ground_truth, predictions_per_model)
        
        processed_count += 1  # Увеличиваем счетчик обработанных документов
    
    return metrics, speed_data


Список моделей

In [279]:
services_and_models = [
    {"service": "openai-proxy", "model": "gpt-4o-mini", "input_cost": 0.02, "output_cost": 0.1},
    {"service": "yandexgpt", "model": "yandexgpt-lite/latest", "input_cost": 0.2, "output_cost": 0.2},
]



Старт основного расчета

In [283]:
from nerus import load_nerus

nerus_path = os.path.join(PATH, "nerus_lenta.conllu.gz")
docs = load_nerus(nerus_path)

metrics, speed_data = run_evaluation(docs, services_and_models)


1725565419.384989
openai-proxy gpt-4o-mini Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.
<Response [200]>
['Голикова Татьяна']
1725565421.1273727
1725565421.133221
yandexgpt yandexgpt-lite/latest Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще в

Подсчет метрик

In [285]:
import pandas as pd
from typing import List, Dict

# Функция для расчёта стоимости
def calculate_cost(speed_data: List[Dict], services_and_models: List[Dict]) -> pd.DataFrame:
    cost_df = pd.DataFrame(services_and_models)
    speed_df = pd.DataFrame(speed_data)

    # Объединение данных о скорости и метриках
    merged_df = pd.merge(speed_df, cost_df, on=['service', 'model'])

    # Рассчёт стоимости
    merged_df['cost'] = (
        merged_df['input_tokens'] * merged_df['input_cost'] + 
        merged_df['output_tokens'] * merged_df['output_cost']
    )

    return merged_df

# Создаём DataFrame из данных
metrics_df = pd.DataFrame(metrics)
speed_df = pd.DataFrame(speed_data)
# Объединяем данные о скорости и метриках
metrics_avg = metrics_df.groupby(['service', 'model']).mean().reset_index()
speed_avg = speed_df.groupby(['service', 'model']).mean().reset_index()
# Переименовываем столбцы перед объединением для избежания конфликтов
speed_avg = speed_avg.rename(columns={
    'processing_time': 'speed_processing_time',
    'input_tokens': 'speed_input_tokens',
    'output_tokens': 'speed_output_tokens'
})

# Объединение с данными о стоимости
costs_df = calculate_cost(speed_data, services_and_models)
merged_df = pd.merge(speed_avg, metrics_avg, on=['service', 'model'])
merged_df = pd.merge(merged_df, costs_df, on=['service', 'model'])

# Поиск лучших моделей по каждой метрике
best_precision = merged_df.loc[merged_df['precision'].idxmax()]
best_recall = merged_df.loc[merged_df['recall'].idxmax()]
best_f1 = merged_df.loc[merged_df['f1'].idxmax()]

# Вывод лучших моделей по каждой метрике
print(f"Best Model for Precision: \n{best_precision}")
print(f"Best Model for Recall: \n{best_recall}")
print(f"Best Model for F1 Score: \n{best_f1}")

# Вывод стоимости работы
print("\nCost of Processing:")
print(merged_df[['service', 'model', 'cost']])

# Дополнительно, вывод средних значений по метрикам и скорости
print("\nAverage Metrics and Speed:")
print(merged_df)


Best Model for Precision: 
service                  openai-proxy
model                     gpt-4o-mini
speed_processing_time        1.942026
speed_input_tokens               99.6
speed_output_tokens              99.6
precision                    0.491667
recall                            1.0
f1                           0.565238
processing_time              1.742384
input_tokens                       92
output_tokens                      92
input_cost                       0.02
output_cost                       0.1
cost                            11.04
Name: 0, dtype: object
Best Model for Recall: 
service                  openai-proxy
model                     gpt-4o-mini
speed_processing_time        1.942026
speed_input_tokens               99.6
speed_output_tokens              99.6
precision                    0.491667
recall                            1.0
f1                           0.565238
processing_time              1.742384
input_tokens                       92
output_tokens 